<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jrossthomson/scientific-computing-examples/blob/main/tutorials/batch_hello_world.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank"  href="https://github.com/jrossthomson/Cottontail/blob/main/batch.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" download="gromacs-slurm.ipynb" href="https://raw.githubusercontent.com/jrossthomson/scientific-computing-examples/main/tutorials/gromacs-on-slurm/gromacs-slurm.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

# Overview
This tutorial will demonstrate how to run some simple jobs using [Google Cloud Batch](https://cloud.google.com/batch/docs/get-started). A few key points:

 1. You have the choice of using a [project you have already created](https://cloud.google.com/resource-manager/docs/creating-managing-projects), or creating one here.
 1. You use a [existing GCS bucket](https://cloud.google.com/storage/docs/creating-buckets#create_a_new_bucket) from your existing project, or you  can create a new one here.
 1. Most of the work communicating with Google Cloud is done with the [gcloud command](https://cloud.google.com/sdk/gcloud)
 1. There are no VMs to provision. All that is done automatically for you by Batch.

The following series of steps will take you through some of the basics of using Cloud Batch.

 > **IMPORTANT:** 
 
It is assumed that the system hosting this notebook has `gcloud` installed. If not, [it can be installed according to these instructions](https://cloud.google.com/sdk/docs/install)
  For this reason, we recommend the use of [Google Colab](https://research.google.com/colaboratory/)



# Getting Started
There are several steps to configure your settings with Google Cloud. Except for the Python installation below, all are completed with the `gcloud` command.
## Run Pip to install packages
Some Python packages are not preinstalled. We add them here.

In [ ]:
%%capture
#@title Pip Install Packages: Run Me! {display-mode: "form"}
!pip install Jinja2
!pip3 install grpcio

## Build Input UI. 

The form below will is for you to enter the required (and optional) information to make the rest of the tutorial work. You will enter:

1. [The project ID](https://cloud.google.com/resource-manager/docs/creating-managing-projects#before_you_begin)
1. You user ID, which is the login id you used to connect to the Google Cloud Console.
1. The [region where you want the jobs](https://cloud.google.com/compute/docs/regions-zones) to run.
1. The name of you [GCS Bucket](https://cloud.google.com/storage/docs/creating-buckets), either already created or the one to be created in your project.
1. Your [billing id](https://cloud.google.com/billing/docs/how-to/view-linked), you may have to ask your organization manager. This is required only if you plan to create a new project here.

Run the next code cell to create the data entry form.

In [ ]:
#@title Form Setup: Run Me! {display-mode: "form"}
#@markdown Run this section to enter your information into the notebook

import ipywidgets as widgets
import subprocess
account = subprocess.run(['gcloud', 'config', 'get-value', 'account'], stdout=subprocess.PIPE)
if not account.stdout: 
   account.stdout = 'enter-user-id' 

project = subprocess.run(['gcloud', 'config', 'get-value', 'project'], stdout=subprocess.PIPE)
if not project.stdout: 
   project.stdout = 'enter-project-id' 

region = subprocess.run(['gcloud',  'config',  'get-value',  'compute/region'],  stdout=subprocess.PIPE)
if not region.stdout: 
   region.stdout = 'us-east1' 

bucket_name = "enter-bucket-name"
billing_id = "enter-billing-id"


project = widgets.Text( value=project.stdout, disabled=False )
username = widgets.Text( value=account.stdout, disabled=False )
region = widgets.Text( value=region.stdout, disabled=False )
bucket = widgets.Text( value=bucket_name, disabled=False )
billing = widgets.Text( value=billing_id, disabled=False )



accordion = widgets.Accordion(children=[project, username, region, bucket, billing])
accordion.set_title(0, 'ProjectID')
accordion.set_title(1, 'UserID')
accordion.set_title(2, 'Region')
accordion.set_title(3, 'Bucket Name')
accordion.set_title(4, 'Billing ID [Optional]')
accordion.selected_index = None
display(accordion)


## Authenticate to Google Cloud
This step allows you to connect and run `gcloud` commands. If you get a <span style="color:yellow">"WARNING"</span>, you may already be logged in.

Follow the instructions to ensure the login process is completed.


In [ ]:
!gcloud auth login $username.value

## Create the project
If the billing ID and the Project ID are set above, you can click here to create the project. If the project already exists, you will get a warning, but it will still work.


In [ ]:
def create_project():
    proj = project.value.rstrip("\n\r ") ## remove "\n", "\r" char in the project value var

    # Don't do project creationg if the billing ID is not set
    if billing.value == "enter-billing-id":
      print("Billing ID not set")
    else:
      !gcloud projects create $proj
      !gcloud beta billing projects link $proj --billing-account=$billing.value
      
create_project()

In [ ]:
def create_bucket():
    proj = project.value.rstrip("\n\r ") ## remove "\n", "\r" char in the project value var
    buck = bucket.value.rstrip("\r\n ") ## remove "\n" char in the bucket value var

    !echo "Executing: "  gcloud alpha storage buckets create gs://$buck --project=$proj
    !gcloud alpha storage buckets create gs://$buck --project=$proj
      
create_bucket()

## Create gcloud configuration

There are two things happening here:
1. You are setting the configurations for `gcloud` defaults
1. You are enabling Google Cloud APIs
1. Finally, the full configuration is listed out.

It is important that these happen in the right order, for example, you have to enable the `compute` API before `gcloud` lets you set a variable for `compute`.

This code cell takes about 30 seconds to execute.

In [ ]:
# set project
!gcloud config set project $project.value 
#enable APIs
!gcloud services enable compute.googleapis.com 
!gcloud services enable batch.googleapis.com
!gcloud services enable monitoring.googleapis.com
# set region 
!gcloud config set compute/region $region.value

# list out the complete configuation
!gcloud config list

# Running jobs
Now that Google Cloud is configured, it is time to create the job files to guide execution, then run the jobs.

## Create job file
The job files are JSON based and contain all the required information to run the tasks described.  The jobs file you will run here are simple. 

The next code cell creates a (somewhat complicated) data structure to contain the raw text of the job files. Essentially, there are two things happening:

1. The JSON is written as a template, so things like the GCS Bucket name can be inserted into the job file. 
1. The JSON template is tied to a list that we use to make the nice green buttons below.

If you want to see the details of the JSON< you can click the "Show code" link and you can see all everything.

Otherwise, just execute the cell and move to the next section.

In [ ]:
#@title Create Templates for JSON settings files  {display-mode: "form"}
#@markdown [These jobs files are from this doc](https://cloud.google.com/batch/docs/get-started#create-basic-job)
#@markdown * Create a JSON file for writing GCS bucket

def create_jobtypelist():
    jobtypelist = {}
    gcs_template = """
    {
        "taskGroups":[
            {
            "taskSpec":{
                "runnables": [
                {
                    "script": {
                    "text": "echo Hello world from task ${BATCH_TASK_INDEX}. >> /mnt/share/log_${JOB_ID}_${BATCH_TASK_INDEX}.txt"
                    },
                    "environment": {
                        "variables": {
                          "JOB_ID": "{{ job_id }} "
                        }
                    }
                  } 
                ],
                "volumes": [
                {
                    "gcs": {
                    "remotePath": "{{ bucket_name }}"
                    },
                    "mountPath": "/mnt/share"
                }
                ]
            },
            "taskCount": 3
            }
        ]
    }

    """

    #@markdown * Create a JSON file for running Hello World using a container
    hello_container_template = """{
        "taskGroups": [
            {
            "taskSpec": {
                "runnables": [
                {
                    "container": {
                    "imageUri": "gcr.io/google-containers/busybox",
                    "entrypoint": "/bin/sh",
                    "commands": [
                        "-c",
                        "echo Hello world! This is task ${BATCH_TASK_INDEX} from JOB ={{ job_id }}. This job has a total of ${BATCH_TASK_COUNT} tasks."
                    ]
                    }
                }
                ],
                "computeResource": {
                "cpuMilli": 2000,
                "memoryMib": 16
                },
                "maxRetryCount": 2,
                "maxRunDuration": "3600s"
            },
            "taskCount": 4,
            "parallelism": 2
            }
        ],
        "allocationPolicy": {
            "instances": [
            {
                "policy": {"machineType": "e2-standard-4"}
            }
            ]
        },
        "labels": {
            "department": "finance",
            "env": "testing"
        },
        "logsPolicy": {
            "destination": "CLOUD_LOGGING"
        }
    }
    """
    jobtypelist["hello"] = {"desc": "Create Hello World", "code" : hello_container_template}
    jobtypelist["gcs"] = {"desc": "Create GCS", "code": gcs_template }

    return(jobtypelist)


# Submit a job
All the preamble was to get you to this point: running a job!

There are a couple of jobs to choose from. 

1. Create files on GCS "Create GCS"
1. Use a container to say "Hello World!": "Create Hello World"

Choosing the appropriate button will submit the job.

* Since job names have to be unique in Batch, a random job id is generated and used to create output file.
* In sections below you will get experience with jobs, listing them and deleting them.

Now, run the next cell and click on a green button.

In [ ]:
#@title Generate buttons to submit jobs {display-mode: "form"}
#@markdown Create a job by selecting the button.
import jinja2
import sys
from os import environ
import ipywidgets as widgets
import uuid

jobid = ""
environment = jinja2.Environment()

jobtypelist = create_jobtypelist()

class MyButton(widgets.Button):
    code = ""

def run_job(b):
    jobid = uuid.uuid4().hex[:8]
    template = environment.from_string(b.code)
    render = template.render(bucket_name=bucket.value, job_id=jobid)
    f = open(jobid + ".json", "w")
    f.write(render)
    f.close()
    out2.clear_output()
    with out2:
        print("\n\nCreating job")
        print("\n\nCreating job file: ", jobid + "txt")
        jobstring = !echo gcloud beta batch jobs submit {jobid} --location us-east1 --config {jobid}.json
        print("Executing command: " + jobstring[0])
        # print(render)
        
        ## This is the command line that is being executed.
        !gcloud beta batch jobs submit {jobid} --location us-east1 --config {jobid}.json

out = widgets.Output(layout={'border': '1px solid black'})
display(out)
out2 = widgets.Output(layout={'border': '1px solid black'})
display(out2)
with out:
    for jobtype in jobtypelist:
        button = MyButton(description=jobtypelist[jobtype]["desc"],  button_style='success')
        button.code = jobtypelist[jobtype] = jobtypelist[jobtype]["code"]
        display(button)
        button.on_click(run_job)



## List jobs 

Once you have submitted jobs above (you can submit as many as you like), you can see the list of jobs 

You can also see the list of jobs in the console UI: 

*  [Job list](https://console.cloud.google.com/batch/jobs)

In [ ]:
!gcloud beta batch jobs list --location $region.value

## Delete jobs
Once a job has run, it will remain in the list of jobs for that project indefinitely. However, if you want to delete a job, you can use the `gcloud` command as:
```
gcloud beta batch jobs delete job_name --location Region
```
For you convenience, you can run the cell below and use the button to delete you jobs by entering the job name.

In [ ]:
#@title Use this form to delete jobs file {display-mode: "form"}
#@markdown [From this doc](https://cloud.google.com/batch/docs/get-started#delete_a_job)

import ipywidgets as widgets

job = widgets.Text( value="f4b6c517", disabled=False )
button = widgets.Button(
    value=False,
    description='Delete job',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
display(job, button)
def delete_me(b):

    ## this is the Gcloud command you will want to execute if deleting manually
    !gcloud beta batch jobs delete $job.value --location $region.value

button.on_click(delete_me)

## See output of the "Hello World" job
If you ran the "Hello World" job, the output was sent to the Cloud Logging system. To view logs, run [`gcloud logging`](https://cloud.google.com/sdk/gcloud/reference/logging) commands.
 
 The next cell provides a sample of how to see the output of the last few jobs

In [ ]:
!gcloud logging read "Hello world" --limit 100 | grep 'This is task [0-9]'

## See output of the "GCS" job
If you ran the "GCS" job, the output was sent to the bucket you created above. 
 
 The next cell provides a sample of how to see the output of all the outputs stored in the GCS bucket.

In [ ]:
!gsutil ls gs://$bucket.value/*.txt
!gsutil cat gs://$bucket.value/*.txt